<a href="https://colab.research.google.com/github/Me-RockingAbhi/My-Projects/blob/master/twitter_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/gdive")

Drive already mounted at /content/gdive; to attempt to forcibly remount, call drive.mount("/content/gdive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
data=pd.read_csv("/content/gdive/My Drive/twitter_train.csv")
data1=pd.read_csv("/content/gdive/My Drive/twitter_test.csv")
data.head()
print(data.shape)

(3235, 6)


In [4]:
data1.shape

(1387, 5)

In [5]:
data.isnull().sum()

id                 0
original_text      0
lang               4
retweet_count      4
original_author    0
sentiment_class    0
dtype: int64

In [6]:
data['lang'].value_counts()
data['lang'].dtypes

dtype('O')

In [0]:
#checking the tweets of languages that are numerical or nan
l=[]
for i in data['lang']:
  if(i!='en'):
    l.append(i) 
for i in l:
  print(data.loc[data['lang'] == i, 'original_text'].iloc[0])
#so,all of the tweets were in english!!

In [0]:
#checking the sentiment class for lang that are numerical or nan
for i in l:
  print(data.loc[data['lang']==i,'sentiment_class'].iloc[0])
#most of the,or nearly all of the lang with numerical are from sentiment class '0'

In [0]:
l1=[]
for i in data['lang']:
  if(i!='en'):
    l1.append(2)
  else:
    l1.append(1)
data['lang']=l1

In [9]:
data['retweet_count'].nunique()
l=[]
for i in data['retweet_count']:
  if(i!=0 or i!=1):
    l.append(i)
for i in l:
  print(data.loc[data['retweet_count']==i,'sentiment_class'].iloc[0])

0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1
0
0
0
0
0
0
0
0
-1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
-1


IndexError: ignored

In [0]:
l=[]
for i in data['retweet_count']:
  if(i!=0 or i!=1):
    l.append(0.5)
  else:
    l.append(int(i))
data['retweet_count']=l;

In [0]:
l1=[]
for i in data1['lang']:
  if(i!='en'):
    l1.append(2)
  else:
    l1.append(1)
data1['lang']=l1

In [0]:
l=[]
for i in data1['retweet_count']:
  if(i!=0 or i!=1):
    l.append(0.5)
  else:
    l.append(int(i))
data1['retweet_count']=l;

In [14]:
print(data1.isnull().sum())
print(data.isnull().sum())
ytr=data['sentiment_class']
data=data.append(data1)

id                 0
original_text      0
lang               0
retweet_count      0
original_author    0
dtype: int64
id                 0
original_text      0
lang               0
retweet_count      0
original_author    0
sentiment_class    0
dtype: int64


In [0]:
data.head()
data=data.drop('sentiment_class',axis=1)
data.dtypes

In [16]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
def cleaned_data(tweets):
  tweets=tweets.lower()
  tokenizer=RegexpTokenizer(r'\w+')
  token=tokenizer.tokenize(tweets)
  stopword=set(stopwords.words('english'))
  new_token=[token for token in token if token not in stopword]
  ps=PorterStemmer()
  stem_token=[ps.stem(token)for token in new_token]
  clean_review=' '.join(stem_token)
  return(clean_review)
data['original_text']=[cleaned_data(i) for i in data['original_text']]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tfidf_vec = TfidfVectorizer(stop_words='english',max_features=50000)
text= tfidf_vec.fit_transform(data['original_text'].values.tolist())
text= pd.DataFrame(text.toarray(), columns=tfidf_vec.get_feature_names())


In [0]:

from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data['original_author']=le.fit_transform(data['original_author'])
data.dtypes
data=data.drop("original_text",axis=1)

In [19]:
train_d=data[:3235]
test_d=data[3235:]
print(train_d.shape)
print(test_d.shape)

(3235, 4)
(1387, 4)


In [20]:
test_d.isnull().sum()

id                 0
lang               0
retweet_count      0
original_author    0
dtype: int64

In [21]:
test_tr=text[3235:]
train_tr=text[:3235]
train_d=pd.concat([train_d,train_tr],axis=1)
train_d.shape

(3235, 17268)

In [22]:
test_d.isnull().sum()
test_tr.reset_index(drop=True, inplace=True)
test_d=pd.concat([test_tr,test_d],axis=1)
test_d.isnull().sum()

00                 0
000                0
0066               0
00awaccvwr         0
00gmrj             0
                  ..
𝒎𝒂𝒎𝒂𝒔              0
id                 0
lang               0
retweet_count      0
original_author    0
Length: 17268, dtype: int64

In [23]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
gbrt=GradientBoostingClassifier(n_estimators=85,max_depth=4)
regr = cross_val_score(gbrt,train_d,ytr,cv=4)
print(regr.mean())

0.5041702872388598


In [0]:
test_d.isnull().sum()

In [0]:
gbrt.fit(train_d,ytr)
ans=gbrt.predict(test_d)


In [0]:
new=pd.DataFrame()
new['id']=data1['id']
new['sentiment_class']=ans
new.to_csv("new2.csv",index=False)